In [1]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

/Users/mamoudou/Developer/Python/ProjetTrutoré/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# CONSTS

In [2]:
CHROME_DRIVER_PATH = "./chromedriver"
BRAVE_PATH = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
DOWNLOAD_DIRECTORY = "/Users/mamoudou/Developer/Python/ProjetTrutoré/"
ARCHIVE_DIRECTORY = "/Users/mamoudou/Developer/Python/ProjetTrutoré/archive"
URL = "https://donneespubliques.meteofrance.fr/?fond=produit&id_produit=90&id_rubrique=32"

## Browser Configs

In [3]:
options = Options()
options.binary_location = BRAVE_PATH  

prefs = {
    "download.default_directory": DOWNLOAD_DIRECTORY,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True 
}
options.add_experimental_option("prefs", prefs)

In [4]:
service = Service(executable_path=CHROME_DRIVER_PATH)

driver = webdriver.Chrome(service=service, options=options)



driver.get(URL)
print("Page Title:", driver.title)
telecharger_button = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Télécharger']")
telecharger_button.click()

time.sleep(5)
driver.quit()

Page Title: Données Publiques de Météo-France - Données SYNOP essentielles OMM


## Fixing the file extension problem

In [5]:
from file_management import change_file_extension

cvs_file = change_file_extension(DOWNLOAD_DIRECTORY)

Le fichier a été renommé : /Users/mamoudou/Developer/Python/ProjetTrutoré/synop.2024122721.txt -> /Users/mamoudou/Developer/Python/ProjetTrutoré/synop.2024122721.csv


# Starting the ELT

In [6]:
import pandas as pd
import numpy as np
df = pd.read_csv(cvs_file, sep=";", dtype={"date": str})

print(df.to_string())

    numer_sta            date    pmer  tend cod_tend   dd    ff           t          td    u     vv  ww  w1  w2    n nbas  hbas  cl  cm  ch    pres niv_bar geop tend24        tn12        tn24        tx12        tx24     tminsol  sw  tw      raf10     rafper  per etat_sol  ht_neige    ssfrai perssfrai        rr1        rr3        rr6       rr12       rr24 phenspe1 phenspe2 phenspe3 phenspe4 nnuage1 ctype1 hnuage1 nnuage2 ctype2 hnuage2 nnuage3 ctype3 hnuage3 nnuage4 ctype4 hnuage4  Unnamed: 59
0        7005  20241227210000  103220   -50        6  120   1.0  273.850000  273.450000   97   7010   0  mq  mq   mq   mq    mq  mq  mq  mq  102300      mq   mq   -280          mq          mq          mq          mq  274.350000  mq  mq   1.800000   3.100000  -10        0  0.000000        mq        mq   0.000000   0.000000   0.000000   0.000000   0.000000       mq       mq       mq       mq      mq     mq      mq      mq     mq      mq      mq     mq      mq      mq     mq      mq          NaN
1   

# Récupération des informations importantes à savoir :
1. Indicatif OMM station : numéro de station `numer_sta` -> `int`
2. Date (UTC) : `date` -> `datetime`
3. Pression au niveau mer : `pmer` -> `int`
4. Variation de pression en 3 heures : `tend` -> `int`
5. Type de tendance barométrique : `cod_tend` -> `int`
6. Direction du vent moyen 10 mn : `dd` -> `int`
7. Vitesse du vent moyen 10 mn : `ff` -> `float`
8. Température : `t` -> `float`
9. Point de rosée : `td` -> `float`
10. Humidité : `u` -> `int`
11. Visibilité horizontale : `vv` -> `float`
12. Temps présent : `ww` -> `int`
13. Nébulosité totale : `n` -> `float`
14. Nébulosité des nuages de l'étage inférieur : `nbas` -> `int`
15. Hauteur de la base des nuages de l'étage inférieur : `hbas` -> `int`
16. Pression station : `pres` -> `int`
17. Variation de pression en 24 heures : `tend24` -> `int`
18. Température minimale sur N heures : `tn12` -> `float`
19. Température maximale sur N heures : `tx12` -> `float`
20. Température minimale du sol sur 12 heures : `tminsol` -> `float`
21. Rafales sur les 10 dernières minutes : `raf10` -> `float`
22. Rafales sur une période : `rafper` -> `float`
23. Période de mesure des rafales : `per` -> `float`
24. Précipitations dans les N dernières heures : `rr12` -> `float`

In [7]:
df = df[['numer_sta', 'date', "pmer", "tend", "cod_tend", "dd", "ff", "t", "td", "u", "vv", "ww", "n", "nbas", "hbas","pres", "tend24", "tn12", "tx12", "tminsol", "raf10", "rafper", "per", "rr12" ]]


print(df.to_string())


    numer_sta            date    pmer  tend cod_tend   dd    ff           t          td    u     vv  ww    n nbas  hbas    pres tend24        tn12        tx12     tminsol      raf10     rafper  per       rr12
0        7005  20241227210000  103220   -50        6  120   1.0  273.850000  273.450000   97   7010   0   mq   mq    mq  102300   -280          mq          mq  274.350000   1.800000   3.100000  -10   0.000000
1        7015  20241227210000  103270   -30        8   90   1.7  272.650000  272.650000  100     80  49  101    9    30  102670   -260          mq          mq  274.750000   2.400000   3.300000  -10  -0.100000
2        7020  20241227210000  103110   -10        0  100   4.0  278.550000  277.950000   96     mq  mq   mq   mq    mq  103000   -200          mq          mq          mq   5.400000   5.400000  -10   0.000000
3        7027  20241227210000  103180   -10        8  160   2.1  275.850000  275.750000   99    160  45  101    9    30  102330   -200          mq          mq  280.

## Remplacer les valeurs manquantes `mq` par `NaN`

In [8]:
df.replace("mq", np.nan, inplace=True)
print(df.to_string())

    numer_sta            date    pmer  tend cod_tend   dd    ff           t          td    u     vv   ww    n nbas  hbas    pres tend24        tn12        tx12     tminsol      raf10     rafper  per       rr12
0        7005  20241227210000  103220   -50        6  120   1.0  273.850000  273.450000   97   7010    0  NaN  NaN   NaN  102300   -280         NaN         NaN  274.350000   1.800000   3.100000  -10   0.000000
1        7015  20241227210000  103270   -30        8   90   1.7  272.650000  272.650000  100     80   49  101    9    30  102670   -260         NaN         NaN  274.750000   2.400000   3.300000  -10  -0.100000
2        7020  20241227210000  103110   -10        0  100   4.0  278.550000  277.950000   96    NaN  NaN  NaN  NaN   NaN  103000   -200         NaN         NaN         NaN   5.400000   5.400000  -10   0.000000
3        7027  20241227210000  103180   -10        8  160   2.1  275.850000  275.750000   99    160   45  101    9    30  102330   -200         NaN         NaN 

## Nettoyages de colonnes non renseignées 

1. Informations générales
	-	Numéro de station : `numer_sta` -> int
	-	Date (UTC) : `date` -> datetime
2. Pression
	-	Pression au niveau mer : `pmer` -> int
	-	Variation de pression en 3 heures : `tend` -> int
	-	Type de tendance barométrique : `cod_tend` -> int
	-	Pression station : `pres` -> int
	-	Variation de pression en 24 heures : `tend24` -> int
3. Vent
	-	Direction du vent moyen 10 mn : `dd` -> int
	-	Vitesse du vent moyen 10 mn : `ff` -> float
	-	Rafales sur les 10 dernières minutes : `raf10` -> float
	-	Rafales sur une période : `rafper` -> float
	-	Période de mesure des rafales : `per` -> float
4. Température
	-	Température : `t` -> float
	-	Point de rosée : `td` -> float
	-	Température minimale sur N heures : `tn12` -> float
	-	Température maximale sur N heures : `tx12` -> float
	-	Température minimale du sol sur 12 heures : `tminsol` -> float
5. Humidité
	-	Humidité : `u` -> int
6. Visibilité
	-	Visibilité horizontale : `vv` -> float
7. Nébulosité et nuages
	-	Nébulosité totale : `n` -> float
	-	Nébulosité des nuages de l’étage inférieur : `nbas` -> int
	-	Hauteur de la base des nuages de l’étage inférieur : `hbas` -> int
8. Précipitations
	-	Précipitations dans les N dernières heures : `rr12` -> float
9. Temps présent
	-	Temps présent : `ww` -> int

In [9]:
df[['dd', 'ff', 'raf10', 'rafper', 'rr12', "per"]] = df[['dd', 'ff', 'raf10', 'rafper', 'rr12', "per"]].fillna(0)

moyenne = ["pmer", "tend", "cod_tend", "pres", "tend24", "t", "td", "tn12", "tx12", "tminsol", "u", "vv", "n", "nbas", "hbas", "ww"]
df[moyenne] = df[moyenne].apply(pd.to_numeric, errors='coerce')

for col in moyenne:
    df[col] = df[col].fillna(df[col].mean())

print(df.to_string())

    numer_sta            date           pmer        tend  cod_tend   dd    ff           t          td           u            vv    ww           n   nbas         hbas           pres       tend24    tn12    tx12     tminsol      raf10     rafper  per       rr12
0        7005  20241227210000  103220.000000  -50.000000  6.000000  120   1.0  273.850000  273.450000   97.000000   7010.000000   0.0   87.869565  4.325   616.521739  102300.000000  -280.000000  268.35  272.45  274.350000   1.800000   3.100000  -10   0.000000
1        7015  20241227210000  103270.000000  -30.000000  8.000000   90   1.7  272.650000  272.650000  100.000000     80.000000  49.0  101.000000  9.000    30.000000  102670.000000  -260.000000  268.35  272.45  274.750000   2.400000   3.300000  -10  -0.100000
2        7020  20241227210000  103110.000000  -10.000000  0.000000  100   4.0  278.550000  277.950000   96.000000  14496.888889  14.5   87.869565  4.325   616.521739  103000.000000  -200.000000  268.35  272.45  276.47222

## Typages des colonnes

In [10]:
df["date"] = pd.to_datetime(df["date"], format='%Y%m%d%H%M%S')
df['date'] = df['date'].dt.strftime("%Y-%m-%d %H:%M:%S")

entier = ["numer_sta", "pmer", "tend", "cod_tend", "dd", "u", "ww", "nbas", "hbas", "pres", "tend24"]
for i in entier:
    print(f"{i} -> {df[col].dtype}")
    df[entier] = df[entier].astype(int)
    print(f"{i} -> {df[col].dtype}")
    print("")

double = ['dd', 'ff', 'raf10', 'rafper', 'rr12', "per", "t", "td", "tn12", "tx12", "tminsol", "n", "vv"]
for i in double:
    print(f"{i} -> {df[col].dtype}")
    df[double] = df[double].astype(float)
    print(f"{i} -> {df[col].dtype}")
    print("")

numer_sta -> float64
numer_sta -> int64

pmer -> int64
pmer -> int64

tend -> int64
tend -> int64

cod_tend -> int64
cod_tend -> int64

dd -> int64
dd -> int64

u -> int64
u -> int64

ww -> int64
ww -> int64

nbas -> int64
nbas -> int64

hbas -> int64
hbas -> int64

pres -> int64
pres -> int64

tend24 -> int64
tend24 -> int64

dd -> int64
dd -> int64

ff -> int64
ff -> int64

raf10 -> int64
raf10 -> int64

rafper -> int64
rafper -> int64

rr12 -> int64
rr12 -> int64

per -> int64
per -> int64

t -> int64
t -> int64

td -> int64
td -> int64

tn12 -> int64
tn12 -> int64

tx12 -> int64
tx12 -> int64

tminsol -> int64
tminsol -> int64

n -> int64
n -> int64

vv -> int64
vv -> int64



In [11]:
print(df.to_string())

    numer_sta                 date    pmer  tend  cod_tend     dd    ff           t          td    u            vv  ww           n  nbas  hbas    pres  tend24    tn12    tx12     tminsol  raf10  rafper   per  rr12
0        7005  2024-12-27 21:00:00  103220   -50         6  120.0   1.0  273.850000  273.450000   97   7010.000000   0   87.869565     4   616  102300    -280  268.35  272.45  274.350000    1.8     3.1 -10.0   0.0
1        7015  2024-12-27 21:00:00  103270   -30         8   90.0   1.7  272.650000  272.650000  100     80.000000  49  101.000000     9    30  102670    -260  268.35  272.45  274.750000    2.4     3.3 -10.0  -0.1
2        7020  2024-12-27 21:00:00  103110   -10         0  100.0   4.0  278.550000  277.950000   96  14496.888889  14   87.869565     4   616  103000    -200  268.35  272.45  276.472222    5.4     5.4 -10.0   0.0
3        7027  2024-12-27 21:00:00  103180   -10         8  160.0   2.1  275.850000  275.750000   99    160.000000  45  101.000000     9    30  

## Conversion des températures de Kelvin en Celcuis

In [12]:
celcuis = ["t", "td", "tn12", "tx12", "tminsol"]
for i in celcuis:
    df[i] = df[i] - 273.15

print(df.to_string())

    numer_sta                 date    pmer  tend  cod_tend     dd    ff          t         td    u            vv  ww           n  nbas  hbas    pres  tend24  tn12  tx12    tminsol  raf10  rafper   per  rr12
0        7005  2024-12-27 21:00:00  103220   -50         6  120.0   1.0   0.700000   0.300000   97   7010.000000   0   87.869565     4   616  102300    -280  -4.8  -0.7   1.200000    1.8     3.1 -10.0   0.0
1        7015  2024-12-27 21:00:00  103270   -30         8   90.0   1.7  -0.500000  -0.500000  100     80.000000  49  101.000000     9    30  102670    -260  -4.8  -0.7   1.600000    2.4     3.3 -10.0  -0.1
2        7020  2024-12-27 21:00:00  103110   -10         0  100.0   4.0   5.400000   4.800000   96  14496.888889  14   87.869565     4   616  103000    -200  -4.8  -0.7   3.322222    5.4     5.4 -10.0   0.0
3        7027  2024-12-27 21:00:00  103180   -10         8  160.0   2.1   2.700000   2.600000   99    160.000000  45  101.000000     9    30  102330    -200  -4.8  -0.7   7

## Arrondir tous les decimaux à 2chiffres apes la virgules

In [13]:
df[double] = df[double].round(2)
print(df.to_string())

    numer_sta                 date    pmer  tend  cod_tend     dd    ff      t     td    u        vv  ww       n  nbas  hbas    pres  tend24  tn12  tx12  tminsol  raf10  rafper   per  rr12
0        7005  2024-12-27 21:00:00  103220   -50         6  120.0   1.0   0.70   0.30   97   7010.00   0   87.87     4   616  102300    -280  -4.8  -0.7     1.20    1.8     3.1 -10.0   0.0
1        7015  2024-12-27 21:00:00  103270   -30         8   90.0   1.7  -0.50  -0.50  100     80.00  49  101.00     9    30  102670    -260  -4.8  -0.7     1.60    2.4     3.3 -10.0  -0.1
2        7020  2024-12-27 21:00:00  103110   -10         0  100.0   4.0   5.40   4.80   96  14496.89  14   87.87     4   616  103000    -200  -4.8  -0.7     3.32    5.4     5.4 -10.0   0.0
3        7027  2024-12-27 21:00:00  103180   -10         8  160.0   2.1   2.70   2.60   99    160.00  45  101.00     9    30  102330    -200  -4.8  -0.7     7.10    5.2     5.2 -10.0  -0.1
4        7037  2024-12-27 21:00:00  103230   -20       

# Archive file

In [14]:
from file_management import move_file

move_file(cvs_file, ARCHIVE_DIRECTORY)

Le fichier a été déplacé de /Users/mamoudou/Developer/Python/ProjetTrutoré/synop.2024122721.csv à /Users/mamoudou/Developer/Python/ProjetTrutoré/archive/synop.2024122721.csv


## Converting DF to JSON and then to readable dict

In [15]:
from datetime import datetime

In [16]:
import pprint   
import json

json_data = df.to_json(orient='records')
dict_data = json.loads(json_data)

pprint.pprint(dict_data)

[{'cod_tend': 6,
  'date': '2024-12-27 21:00:00',
  'dd': 120.0,
  'ff': 1.0,
  'hbas': 616,
  'n': 87.87,
  'nbas': 4,
  'numer_sta': 7005,
  'per': -10.0,
  'pmer': 103220,
  'pres': 102300,
  'raf10': 1.8,
  'rafper': 3.1,
  'rr12': 0.0,
  't': 0.7,
  'td': 0.3,
  'tend': -50,
  'tend24': -280,
  'tminsol': 1.2,
  'tn12': -4.8,
  'tx12': -0.7,
  'u': 97,
  'vv': 7010.0,
  'ww': 0},
 {'cod_tend': 8,
  'date': '2024-12-27 21:00:00',
  'dd': 90.0,
  'ff': 1.7,
  'hbas': 30,
  'n': 101.0,
  'nbas': 9,
  'numer_sta': 7015,
  'per': -10.0,
  'pmer': 103270,
  'pres': 102670,
  'raf10': 2.4,
  'rafper': 3.3,
  'rr12': -0.1,
  't': -0.5,
  'td': -0.5,
  'tend': -30,
  'tend24': -260,
  'tminsol': 1.6,
  'tn12': -4.8,
  'tx12': -0.7,
  'u': 100,
  'vv': 80.0,
  'ww': 49},
 {'cod_tend': 0,
  'date': '2024-12-27 21:00:00',
  'dd': 100.0,
  'ff': 4.0,
  'hbas': 616,
  'n': 87.87,
  'nbas': 4,
  'numer_sta': 7020,
  'per': -10.0,
  'pmer': 103110,
  'pres': 103000,
  'raf10': 5.4,
  'rafper': 5.

In [17]:
first = dict_data[0]
js = json.dumps(first)

print(js)

{"numer_sta": 7005, "date": "2024-12-27 21:00:00", "pmer": 103220, "tend": -50, "cod_tend": 6, "dd": 120.0, "ff": 1.0, "t": 0.7, "td": 0.3, "u": 97, "vv": 7010.0, "ww": 0, "n": 87.87, "nbas": 4, "hbas": 616, "pres": 102300, "tend24": -280, "tn12": -4.8, "tx12": -0.7, "tminsol": 1.2, "raf10": 1.8, "rafper": 3.1, "per": -10.0, "rr12": 0.0}
